In [ ]:

import cv2 as cv
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

'''
 Tuning tips
Hue bounds: widen or shift (0–10) / (170–180) 
based on your printed percentiles.
S/V floors: if you’re catching too-dark or too-pale areas, 
raise the S and/or V lower thresholds (e.g. S≥80, V≥80).
Kernel size: increase your morph open/close 
for larger fruits—or shrink it to preserve small details.
'''

# ─ CONFIG ─────────────────────────────────────────
'''change the FOLDER NAME AND THE IMAGE NAME'''


BASE_DATE  = "16_7_2025"
FOLDER_NAME = "sample"
IMAGE_NAME = "DJI_0382_zoomed_27.jpg"
BASE_PATH  = Path(f"/Users/at/Orbit_Red_Blob/Data/UKE_Plot_14_DD_{BASE_DATE}/{FOLDER_NAME}")

img_bgr    = cv.imread(str(BASE_PATH / IMAGE_NAME))
if img_bgr is None:
    raise FileNotFoundError(f"Couldn’t load image: {BASE_PATH/IMAGE_NAME}")


# CONVERT TO HSV
hsv = cv.cvtColor(img_bgr, cv.COLOR_BGR2HSV)
H, S, V = cv.split(hsv)


# 1a) Compute simple stats
h_min, h_max = H.min(), H.max()
h_5,  h_95  = np.percentile(H, 5), np.percentile(H, 95)
print("Hue      min/max:", h_min, h_max)
print("Hue  5th/95th pct:", h_5,  h_95)

# 1b) Optionally plot a histogram
import matplotlib.pyplot as plt
plt.hist(H.ravel(), bins=180)
plt.title("Hue distribution")
plt.xlabel("Hue value (0–179)")
plt.show()


# # two ranges because hue wraps around
# ─── HSV THRESHOLDS ──────────────────────────────────────────────────
# lower_red1 = np.array([0, 100, 20])
# upper_red1 = np.array([10,255,150])
# lower_red2 = np.array([160, 100, 25])
# upper_red2 = np.array([179, 255, 188])

# ─── HSV THRESHOLDS 2──────────────────────────────────────────────────
lower_red1 = np.array((0, 50, 50))
upper_red1 = np.array([10, 255, 200])
lower_red2 = np.array([160, 50, 50])
upper_red2 = np.array([179, 255, 200])

mask1 = cv.inRange(hsv, lower_red1, upper_red1)
mask2 = cv.inRange(hsv, lower_red2, upper_red2)
mask  = cv.bitwise_or(mask1, mask2)

kernel = np.ones((5,5), np.uint8)
mask = cv.morphologyEx(mask, cv.MORPH_OPEN,  kernel)  # remove tiny specks
mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)  # fill small holes

highlight = img_bgr.copy()
highlight[mask>0] = (0,0,255)  # pure red paint
cv.imwrite(str(BASE_PATH / "output_HSV_thresholded.jpg"), highlight)
